## 1. Installation and Setup

First, install the required packages:

In [ ]:
# Install Google ADK and dependencies
!pip install google-adk
!pip install google-generativeai

## 2. Import Required Libraries

In [117]:
import os
import json
from datetime import datetime
from typing import Dict, List, Any

from google.adk.agents import LlmAgent
from google.adk.tools.tool_context import ToolContext
from google.adk import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

## 3. Configure API Keys

Set up your Google API credentials:

In [ ]:
# Set your Google API key
os.environ["GOOGLE_API_KEY"] = ""

# Alternatively, load from a .env file or prompt user
from getpass import getpass
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

## 4. Define Custom Tools

### 4.1 Memory Tools

In [177]:
# In-memory storage for user data
user_memory_store: Dict[str, Dict[str, Any]] = {}

# Enhanced storage for therapeutic response tracking
# Tracks what responses work/don't work for specific emotional triggers
therapeutic_patterns: Dict[str, Dict[str, Any]] = {}

def load_memory(tool_context: ToolContext) -> Dict[str, Any]:
    """Load user memory including preferences, history, and personal details.
    
    This tool automatically retrieves the current user's ID from the session context.
        
    Returns:
        A dictionary containing user's stored information.
    """
    # Get user_id from the invocation context (session)
    user_id = tool_context.user_id
    
    memory = user_memory_store.get(user_id, {
        "preferences": {},
        "history": [],
        "personal_details": {},
        "interests": []
    })
    
    # Store in tool context state for other agents to access
    tool_context.state['memory'] = memory
    tool_context.state['user_id'] = user_id
    
    return {"user_id": user_id, "memory": memory}

def save_memory(key: str, value: str, tool_context: ToolContext) -> str:
    """Save user information to memory.
    
    This tool automatically uses the current user's ID from the session context.
    
    Args:
        key: The category to store the information (e.g., 'preferences', 'history', 'personal_details', 'name', 'interests').
        value: The value to store (as a string or JSON string for complex data).
        
    Returns:
        A confirmation message.
    """
    # Get user_id from the invocation context
    user_id = tool_context.user_id
    
    if user_id not in user_memory_store:
        user_memory_store[user_id] = {
            "preferences": {},
            "history": [],
            "personal_details": {},
            "interests": []
        }
    
    # Handle different key types
    if key == "preferences":
        if isinstance(user_memory_store[user_id]["preferences"], dict):
            user_memory_store[user_id]["preferences"]["general"] = value
        else:
            user_memory_store[user_id]["preferences"] = {"general": value}
    elif key == "history":
        user_memory_store[user_id]["history"].append(value)
    elif key == "personal_details":
        if isinstance(user_memory_store[user_id]["personal_details"], dict):
            user_memory_store[user_id]["personal_details"]["info"] = value
        else:
            user_memory_store[user_id]["personal_details"] = {"info": value}
    elif key == "name":
        user_memory_store[user_id]["personal_details"]["name"] = value
    elif key == "interests":
        if "interests" not in user_memory_store[user_id]:
            user_memory_store[user_id]["interests"] = []
        user_memory_store[user_id]["interests"].append(value)
    else:
        # For any other key, store directly
        user_memory_store[user_id][key] = value
    
    # Update tool context state
    tool_context.state['memory'] = user_memory_store[user_id]
    
    return f"Saved '{key}' = '{value}' to memory."


# ============================================================
# THERAPEUTIC PATTERN TRACKING TOOLS
# ============================================================
# These tools track what emotional responses work/don't work for users

def get_therapeutic_patterns(tool_context: ToolContext) -> Dict[str, Any]:
    """Get all stored therapeutic patterns for a user.
    
    Returns patterns of what responses have worked or not worked
    for specific emotional triggers.
    
    Returns:
        Dictionary containing liked/disliked response patterns for triggers.
    """
    user_id = tool_context.user_id
    patterns = therapeutic_patterns.get(user_id, {
        "triggers": {},  # Maps trigger phrases to response history
        "preferred_styles": [],  # List of response styles user likes
        "avoided_styles": []  # List of response styles user dislikes
    })
    return patterns

def save_therapeutic_response(
    trigger: str,
    response: str,
    liked: bool,
    tool_context: ToolContext,
    notes: str = ""
) -> str:
    """Save a therapeutic response pattern - tracks what works/doesn't work.
    
    When a user shares an emotional struggle and reacts to AI's response,
    this tool records whether the response was helpful or not.
    
    Args:
        trigger: The emotional trigger phrase (e.g., "life is hard", "feeling stressed")
        response: The response that was given
        liked: True if user liked/found helpful, False if not
        notes: Optional notes about why it worked or didn't
        
    Returns:
        Confirmation message
    """
    user_id = tool_context.user_id
    
    if user_id not in therapeutic_patterns:
        therapeutic_patterns[user_id] = {
            "triggers": {},
            "preferred_styles": [],
            "avoided_styles": []
        }
    
    # Normalize trigger for matching
    trigger_key = trigger.lower().strip()
    
    if trigger_key not in therapeutic_patterns[user_id]["triggers"]:
        therapeutic_patterns[user_id]["triggers"][trigger_key] = {
            "liked_responses": [],
            "disliked_responses": [],
            "attempt_count": 0
        }
    
    therapeutic_patterns[user_id]["triggers"][trigger_key]["attempt_count"] += 1
    
    response_entry = {
        "response": response,
        "notes": notes,
        "timestamp": datetime.now().isoformat()
    }
    
    if liked:
        therapeutic_patterns[user_id]["triggers"][trigger_key]["liked_responses"].append(response_entry)
        status = "✓ Saved as HELPFUL response"
    else:
        therapeutic_patterns[user_id]["triggers"][trigger_key]["disliked_responses"].append(response_entry)
        status = "✓ Saved as UNHELPFUL response (will avoid similar in future)"
    
    return f"{status} for trigger '{trigger}'"

def get_preferred_response_for_trigger(trigger: str, tool_context: ToolContext) -> Dict[str, Any]:
    """Find if there's a preferred response style for a specific emotional trigger.
    
    Looks up stored patterns to find what has worked before for similar triggers.
    
    Args:
        trigger: The emotional trigger phrase to look up
        
    Returns:
        Dictionary with preferred responses (if any) and styles to avoid
    """
    user_id = tool_context.user_id
    trigger_key = trigger.lower().strip()
    
    result = {
        "found_pattern": False,
        "liked_responses": [],
        "disliked_responses": [],
        "recommendation": ""
    }
    
    if user_id in therapeutic_patterns:
        patterns = therapeutic_patterns[user_id]
        
        # Direct trigger match
        if trigger_key in patterns["triggers"]:
            trigger_data = patterns["triggers"][trigger_key]
            result["found_pattern"] = True
            result["liked_responses"] = trigger_data["liked_responses"]
            result["disliked_responses"] = trigger_data["disliked_responses"]
            
            if trigger_data["liked_responses"]:
                result["recommendation"] = f"Use a response similar to: {trigger_data['liked_responses'][-1]['response']}"
            if trigger_data["disliked_responses"]:
                result["recommendation"] += f" Avoid responses like: {trigger_data['disliked_responses'][-1]['response']}"
        
        # Check for similar triggers (fuzzy matching)
        else:
            for stored_trigger in patterns["triggers"]:
                # Simple substring matching for now
                if trigger_key in stored_trigger or stored_trigger in trigger_key:
                    trigger_data = patterns["triggers"][stored_trigger]
                    result["found_pattern"] = True
                    result["liked_responses"] = trigger_data["liked_responses"]
                    result["disliked_responses"] = trigger_data["disliked_responses"]
                    if trigger_data["liked_responses"]:
                        result["recommendation"] = f"Similar trigger found. Use: {trigger_data['liked_responses'][-1]['response']}"
                    break
    
    return result

### 4.2 Calendar Tool (with MCP Integration Ready)

In [178]:
# In-memory storage for calendar events (fallback when MCP not available)
calendar_events: List[Dict[str, Any]] = []

def get_events(tool_context: ToolContext) -> List[Dict[str, Any]]:
    """Get all scheduled events for the current user.
    
    Returns:
        A list of scheduled events.
    """
    user_id = tool_context.user_id
    user_events = [e for e in calendar_events if e.get("user_id") == user_id]
    return user_events

def schedule_event(
    title: str,
    date: str,
    time: str,
    tool_context: ToolContext,
    description: str = ""
) -> str:
    """Schedule an event or reminder in the calendar.
    
    Args:
        title: The title of the event.
        date: The date of the event (format: YYYY-MM-DD).
        time: The time of the event (format: HH:MM).
        description: Optional description of the event.
        
    Returns:
        A confirmation message with event details.
    """
    user_id = tool_context.user_id
    
    event = {
        "id": len(calendar_events) + 1,
        "user_id": user_id,
        "title": title,
        "date": date,
        "time": time,
        "description": description,
        "created_at": datetime.now().isoformat()
    }
    
    calendar_events.append(event)
    
    # Store in tool context state
    if 'scheduled_events' not in tool_context.state:
        tool_context.state['scheduled_events'] = []
    tool_context.state['scheduled_events'].append(event)
    
    return f"✓ Event '{title}' scheduled for {date} at {time}."

### 4.3 MCP Google Calendar Integration (Optional)

To connect to the real Google Calendar via MCP, you need:
1. Set up the Google Calendar MCP server
2. Configure OAuth credentials

The following cell sets up the MCP toolset for Google Calendar integration.

In [179]:
# MCP Google Calendar Integration
# This provides real Google Calendar access when configured

from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
try:
    from mcp import StdioServerParameters
    MCP_AVAILABLE = True
except ImportError:
    MCP_AVAILABLE = False
    print("⚠️ MCP not installed. Run: pip install mcp")
    print("   Using local calendar tools instead.")

# Flag to control MCP usage
USE_MCP_CALENDAR = False  # Set to True to use real Google Calendar

def get_mcp_calendar_toolset():
    """Create MCP toolset for Google Calendar.
    
    Prerequisites:
    1. pip install mcp
    2. npm install -g @anthropic/google-calendar-mcp (or use npx)
    3. Set up Google OAuth credentials
    
    Returns:
        MCPToolset configured for Google Calendar, or None if not available
    """
    if not MCP_AVAILABLE:
        return None
    
    try:
        mcp_calendar = MCPToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=[
                        "-y",
                        "@anthropic/google-calendar-mcp"
                    ],
                    # Add environment variables for OAuth if needed
                    # env={
                    #     "GOOGLE_CLIENT_ID": os.environ.get("GOOGLE_CLIENT_ID", ""),
                    #     "GOOGLE_CLIENT_SECRET": os.environ.get("GOOGLE_CLIENT_SECRET", ""),
                    # }
                ),
                timeout=30,
            ),
            # Filter to specific calendar tools we want to expose
            tool_filter=['list-events', 'create-event', 'search-events', 'list-calendars']
        )
        return mcp_calendar
    except Exception as e:
        print(f"⚠️ Could not initialize MCP Calendar: {e}")
        return None

# Initialize MCP calendar if enabled
mcp_calendar_toolset = None
if USE_MCP_CALENDAR and MCP_AVAILABLE:
    mcp_calendar_toolset = get_mcp_calendar_toolset()
    if mcp_calendar_toolset:
        print("✓ MCP Google Calendar toolset initialized!")
    else:
        print("⚠️ MCP Calendar initialization failed, using local tools")
else:
    print("ℹ️ Using local calendar tools (set USE_MCP_CALENDAR=True for real Google Calendar)")

ℹ️ Using local calendar tools (set USE_MCP_CALENDAR=True for real Google Calendar)


## 5. Define Sub-Agents

### 5.1 Memory Agent

In [180]:
from google.adk.models.google_llm import Gemini

# Configure retry settings for rate limits
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=2,  # Exponential backoff multiplier
    initial_delay=1,  # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504]  # Retry on these HTTP errors
)

memory_agent = LlmAgent(
    name="memory_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Manages user memory - saves names, interests, preferences, and retrieves user context.",
    instruction="""You are a memory management assistant. You help store and retrieve user information.

IMPORTANT: The user_id is automatically provided by the system - never ask for it.

When you receive a request:
1. If the user introduces themselves or shares info (name, interest, preference):
   - Call save_memory with the appropriate key ('name', 'interests', 'preferences') and value
   - After the tool responds, reply with a friendly confirmation like "Nice to meet you, [name]!" or "I'll remember that you're interested in [topic]!"

2. If asked to recall/load user information:
   - Call load_memory (no parameters needed)
   - Summarize the information you found

ALWAYS respond with a friendly message after using your tools. Never leave a response blank.""",
    tools=[load_memory, save_memory],
    output_key="memory_result"
)

### 5.2 Calendar Agent

In [181]:
# Determine which calendar tools to use
calendar_tools = [schedule_event, get_events]
if mcp_calendar_toolset:
    calendar_tools = [mcp_calendar_toolset]  # Use MCP tools if available
    print("Calendar Agent will use MCP Google Calendar tools")
else:
    print("Calendar Agent will use local calendar tools")

calendar_agent = LlmAgent(
    name="calendar_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Schedules meetings, events, and reminders.",
    instruction="""You are a calendar scheduling assistant.

When you receive a scheduling request:
1. Extract the event details: title, date (YYYY-MM-DD format), and time (HH:MM format)
2. If the user says "tomorrow", calculate the date as today + 1 day
3. If time is not specified, use a reasonable default (e.g., 07:00 for morning, 14:00 for afternoon)
4. Call the schedule_event tool with the extracted details
5. After the tool responds, confirm the scheduled event with a friendly message

IMPORTANT:
- The user_id is automatic - never ask for it
- ALWAYS respond with a friendly confirmation after scheduling
- Today's date context should be inferred from the current time""",
    tools=calendar_tools,
    output_key="calendar_result"
)

Calendar Agent will use local calendar tools


### 5.3 Feedback Agent

In [182]:
feedback_agent = LlmAgent(
    name="feedback_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Processes user feedback about interaction preferences and saves them.",
    instruction="""You are a feedback processing assistant. Your job is to save user preferences.

REQUIRED STEPS when you receive feedback:
1. Extract the preference (e.g., "prefers concise answers", "wants shorter responses")
2. ALWAYS call save_memory with:
   - key: "preferences"
   - value: the extracted preference as a clear statement
3. After the tool confirms saving, acknowledge with a brief message

Example:
- User says: "I prefer shorter answers"
- You MUST call: save_memory(key="preferences", value="prefers shorter, concise answers")
- Then respond: "Got it! I'll keep that in mind."

NEVER skip calling save_memory - always save the preference first.""",
    tools=[save_memory],
    output_key="feedback_result"
)

### 5.4 Therapeutic Response Agent (Learns What Works)

This agent tracks emotional triggers and learns which responses help the user:
- When user shares something bothering them, it provides a response
- If user asks for another option, it provides an alternative
- When user expresses liking/disliking a response, it saves the pattern
- Future interactions use learned patterns to give better responses

In [183]:
therapeutic_agent = LlmAgent(
    name="therapeutic_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Provides supportive responses to emotional struggles and learns what helps each user.",
    instruction="""You are a compassionate therapeutic support assistant who learns what works for each user.

YOUR WORKFLOW:

1. **When user shares something bothering them** (e.g., "I feel my life is so hard"):
   - FIRST: Call get_preferred_response_for_trigger with the emotional trigger
   - If a preferred response exists, use that style/approach
   - If no pattern exists or user wants something new, provide a supportive response
   - Keep track of what you said (you may need to save it later)

2. **When user asks for another response/option**:
   - Provide an ALTERNATIVE perspective, such as:
     * "Remember those less fortunate who still smile and do what they can"
     * "Miracles happen for those who never quit"
     * "Every struggle is preparing you for something greater"
     * "You've overcome challenges before, you can do it again"
   - These alternatives should be different in tone and approach from your first response

3. **When user expresses they LIKE a response** (e.g., "I like that", "that helps"):
   - Call save_therapeutic_response with:
     * trigger: the original emotional trigger
     * response: the response they liked
     * liked: True
   - Acknowledge their preference warmly

4. **When user expresses they DON'T LIKE a response** (e.g., "that doesn't help", "try again"):
   - Call save_therapeutic_response with:
     * trigger: the original emotional trigger  
     * response: the response they didn't like
     * liked: False
   - Offer a different approach

RESPONSE STYLES TO VARY BETWEEN:
- Perspective shift (think of others who have it harder)
- Hope/resilience focus (miracles, never giving up)
- Validation/empathy (acknowledging feelings)
- Practical action (small steps to take)
- Growth mindset (struggles make you stronger)

IMPORTANT:
- Never be preachy or dismissive
- Be genuinely warm and supportive
- Remember: you're learning what helps THIS specific person
- Use their preferred patterns when available""",
    tools=[
        get_therapeutic_patterns,
        save_therapeutic_response,
        get_preferred_response_for_trigger
    ],
    output_key="therapeutic_result"
)

print("✓ Therapeutic Response Agent created!")

✓ Therapeutic Response Agent created!


## 6. Define Root Agent (Personal Psychologist)

The root agent coordinates all sub-agents:

In [184]:
root_agent = LlmAgent(
    name="personal_psychologist",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="An empathetic personal assistant that remembers user context and helps with various tasks.",
    instruction="""You are a friendly personal assistant with specialized sub-agents. You MUST delegate to sub-agents for specific tasks.

MANDATORY DELEGATION RULES:

1. **ALWAYS transfer to memory_agent** when the user:
   - Shares their name (e.g., "I'm Alice", "My name is...")
   - Mentions interests or hobbies
   - Says "remember that..." or asks to save information
   - Asks "what do you know about me?"

2. **ALWAYS transfer to calendar_agent** when the user:
   - Wants to schedule something
   - Mentions dates, times, appointments, reminders
   - Says "schedule", "remind me", "set up a meeting"

3. **ALWAYS transfer to feedback_agent** when the user:
   - Gives feedback about your responses
   - Says "I prefer shorter/longer answers"
   - Comments on communication style

4. **ALWAYS transfer to therapeutic_agent** when the user:
   - Shares emotional struggles ("I feel...", "I'm stressed", "life is hard")
   - Expresses negative emotions or seeks comfort
   - Asks for another perspective or alternative response
   - Says they like or don't like a supportive response

5. **Only handle directly** when:
   - General questions not involving memory, calendar, feedback, or emotional support
   - Casual conversation with no action needed

When a sub-agent returns, incorporate their result naturally. Be warm and never expose internal processes.""",
    sub_agents=[memory_agent, calendar_agent, feedback_agent, therapeutic_agent]
)

print("✓ Root agent and all sub-agents created successfully!")
print(f"  Sub-agents: {[agent.name for agent in root_agent.sub_agents]}")

✓ Root agent and all sub-agents created successfully!
  Sub-agents: ['memory_agent', 'calendar_agent', 'feedback_agent', 'therapeutic_agent']


## 7. Create Runner and Session Service

Set up the runner to execute agent conversations:

In [185]:
# Create session service for managing conversation state
session_service = InMemorySessionService()

# Create runner
runner = Runner(
    app_name="personal_psychologist_app",
    agent=root_agent,
    session_service=session_service
)

print("✓ Runner initialized successfully!")

✓ Runner initialized successfully!


In [108]:
# from google.adk.agents import Agent
# from google.adk.models.google_llm import Gemini
# from google.adk.runners import InMemoryRunner
# from google.adk.tools import google_search

# retry_config=types.HttpRetryOptions(
#     attempts=5,  # Maximum retry attempts
#     exp_base=7,  # Delay multiplier
#     initial_delay=1, # Initial delay before first retry (in seconds)
#     http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
# )

# root_agent = Agent(
#     name="helpful_assistant",
#     model=Gemini(
#         model="gemini-2.5-flash-lite",
#         retry_options=retry_config
#     ),
#     description="A simple agent that can answer general questions.",
#     instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
#     tools=[google_search],
# )

# print("✅ Root Agent defined.")
# runner = InMemoryRunner(agent=root_agent)

# response = await runner.run_debug(
#     "What is Agent Development Kit from Google? What languages is the SDK available in?"
# )

In [109]:
# Note: run_debug shows all intermediate agent responses (duplicates are normal)
# Use the run_query helper function below for cleaner output
response = await runner.run_debug("What's the weather in London?")


 ### Created new session: debug_session_id

User > What's the weather in London?
calendar_agent > I am a calendar assistant and can only schedule events or set reminders. I cannot provide weather information. Please ask me to set a reminder or schedule an event.
calendar_agent > I am a calendar assistant and can only schedule events or set reminders. I cannot provide weather information. Please ask me to set a reminder or schedule an event.


## 8. Helper Function for Running Queries

In [186]:
async def run_query(query: str, user_id: str = "user123", session_id: str = "session456", verbose: bool = False):
    """Run a query through the agent system and display results.
    
    Args:
        query: The user's query or message.
        user_id: The unique user identifier.
        session_id: The session identifier for conversation continuity.
        verbose: If True, show all agent interactions; if False, show only final response.
    """
    print(f"\n{'='*60}")
    print(f"USER: {query}")
    print(f"{'='*60}\n")
    
    # Create session if it doesn't exist
    try:
        session = await session_service.get_session(
            app_name="personal_psychologist_app",
            user_id=user_id,
            session_id=session_id
        )
        if session is None:
            raise Exception("Session not found")
    except:
        session = await session_service.create_session(
            app_name="personal_psychologist_app",
            user_id=user_id,
            session_id=session_id,
            state={}
        )
    
    content = types.Content(
        role='user',
        parts=[types.Part(text=query)]
    )
    
    final_response = ""
    all_responses = []
    
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=content
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    if verbose:
                        print(f"[{event.author}]: {part.text}")
                    all_responses.append({"author": event.author, "text": part.text})
                    # Keep track of the last response from the root agent
                    if event.author == "personal_psychologist":
                        final_response = part.text
                elif hasattr(part, 'function_call') and part.function_call:
                    if verbose:
                        fc = part.function_call
                        args_str = ', '.join(f'{k}={v!r}' for k, v in fc.args.items())
                        print(f"[{event.author}] 📞 Tool Call: {fc.name}({args_str})")
                elif hasattr(part, 'function_response') and part.function_response:
                    if verbose:
                        fr = part.function_response
                        print(f"[{event.author}] ↩️ Tool Response: {fr.name} -> {fr.response}")
    
    # If no root agent response, use the last response from any agent
    if not final_response and all_responses:
        final_response = all_responses[-1]["text"]
    
    print(f"ASSISTANT: {final_response}")
    print(f"\n{'='*60}\n")
    return final_response

## 9. Test the Multi-Agent System

### 9.1 Test Memory Agent

In [195]:
# Test loading and saving memory
await run_query(
    "Hi! My name is Alice and I'm interested in mindfulness and meditation.",
    user_id="alice123",
    session_id="session_alice_1"
)


USER: Hi! My name is Alice and I'm interested in mindfulness and meditation.

ASSISTANT: Nice to meet you, Alice! I'll remember that you're interested in mindfulness and meditation.


ASSISTANT: Nice to meet you, Alice! I'll remember that you're interested in mindfulness and meditation.




"Nice to meet you, Alice! I'll remember that you're interested in mindfulness and meditation."

### 9.2 Test Calendar Agent

In [196]:
# Test scheduling an event
await run_query(
    "Can you schedule a meditation session for me tomorrow at 7 AM?",
    user_id="alice123",
    session_id="session_alice_1"
)


USER: Can you schedule a meditation session for me tomorrow at 7 AM?

ASSISTANT: I've scheduled your meditation session for tomorrow at 07:00.


ASSISTANT: I've scheduled your meditation session for tomorrow at 07:00.




"I've scheduled your meditation session for tomorrow at 07:00."

### 9.3 Test Context Awareness

In [173]:
# Test if the agent remembers previous context
await run_query(
    "What are some good mindfulness practices for beginners?",
    user_id="alice123",
    session_id="session_alice_1"
)


USER: What are some good mindfulness practices for beginners?

ASSISTANT: Mindfulness practices for beginners often focus on simple, accessible techniques. A great starting point is **mindful breathing**. Simply pay attention to the sensation of your breath as it enters and leaves your body. Notice the rise and fall of your chest or abdomen. When your mind wanders, gently guide your attention back to your breath without judgment.

Another accessible practice is a **body scan**. Lie down or sit comfortably and bring your awareness to different parts of your body, from your toes to the top of your head. Notice any sensations – warmth, coolness, tingling, pressure – without trying to change them.

A **walking meditation** can also be very beneficial. As you walk, focus on the physical sensations of your feet touching the ground, the movement of your legs, and the rhythm of your steps.

Finally, **mindful eating** is a wonderful way to integrate mindfulness into daily life. Before you eat

'Mindfulness practices for beginners often focus on simple, accessible techniques. A great starting point is **mindful breathing**. Simply pay attention to the sensation of your breath as it enters and leaves your body. Notice the rise and fall of your chest or abdomen. When your mind wanders, gently guide your attention back to your breath without judgment.\n\nAnother accessible practice is a **body scan**. Lie down or sit comfortably and bring your awareness to different parts of your body, from your toes to the top of your head. Notice any sensations – warmth, coolness, tingling, pressure – without trying to change them.\n\nA **walking meditation** can also be very beneficial. As you walk, focus on the physical sensations of your feet touching the ground, the movement of your legs, and the rhythm of your steps.\n\nFinally, **mindful eating** is a wonderful way to integrate mindfulness into daily life. Before you eat, take a moment to observe your food – its colors, textures, and sme

### 9.4 Test Feedback Agent

In [174]:
# Test feedback processing
await run_query(
    "I really appreciate your detailed responses, but I'd prefer shorter, more concise answers in the future.",
    user_id="alice123",
    session_id="session_alice_1"
)


USER: I really appreciate your detailed responses, but I'd prefer shorter, more concise answers in the future.

ASSISTANT: Got it! I'll keep that in mind.


ASSISTANT: Got it! I'll keep that in mind.




"Got it! I'll keep that in mind."

### 9.5 Test Complex Multi-Agent Interaction

In [175]:
# Test a complex query that might involve multiple agents
await run_query(
    "I've been feeling stressed lately. Can you remind me of my interests and schedule a relaxation session?",
    user_id="alice123",
    session_id="session_alice_1"
)


USER: I've been feeling stressed lately. Can you remind me of my interests and schedule a relaxation session?

ASSISTANT: I can schedule a relaxation session for you. What day and time would work best?


ASSISTANT: I can schedule a relaxation session for you. What day and time would work best?




'I can schedule a relaxation session for you. What day and time would work best?'

### 9.6 Test Therapeutic Agent - Learning Response Patterns

This tests the scenario:
1. User shares something bothering them
2. AI gives first response
3. User asks for another option
4. User expresses which response they prefer
5. Pattern is saved for future use

In [187]:
# Step 1: User shares an emotional struggle
print("=" * 60)
print("THERAPEUTIC AGENT TEST - Learning Response Patterns")
print("=" * 60)

await run_query(
    "I feel like my life is so hard sometimes. Everything seems to be going wrong.",
    user_id="alice123",
    session_id="session_alice_therapy"
)

THERAPEUTIC AGENT TEST - Learning Response Patterns

USER: I feel like my life is so hard sometimes. Everything seems to be going wrong.

ASSISTANT: I understand that you feel like your life is hard and that everything is going wrong. It's completely normal to feel overwhelmed when many things happen at once. Remember that difficult times are temporary, and you have the strength to overcome them. What specifically is making you feel this way? Sharing more details might help us find ways to cope.




"I understand that you feel like your life is hard and that everything is going wrong. It's completely normal to feel overwhelmed when many things happen at once. Remember that difficult times are temporary, and you have the strength to overcome them. What specifically is making you feel this way? Sharing more details might help us find ways to cope."

In [188]:
# Step 2: User asks for another perspective
await run_query(
    "Can you give me another way to think about this?",
    user_id="alice123",
    session_id="session_alice_therapy"
)


USER: Can you give me another way to think about this?

ASSISTANT: When you're facing a lot of challenges, it can be helpful to remember that even in the darkest times, there's always a glimmer of hope. Think of every struggle as a stepping stone, preparing you for something even greater. You've overcome difficulties before, and you have the resilience within you to face this too.




"When you're facing a lot of challenges, it can be helpful to remember that even in the darkest times, there's always a glimmer of hope. Think of every struggle as a stepping stone, preparing you for something even greater. You've overcome difficulties before, and you have the resilience within you to face this too."

In [189]:
# Step 3: User expresses they like this response - save the pattern!
await run_query(
    "I really like that second response - it reminds me that miracles happen for those who never quit. That helps a lot.",
    user_id="alice123",
    session_id="session_alice_therapy"
)


USER: I really like that second response - it reminds me that miracles happen for those who never quit. That helps a lot.

ASSISTANT: That's wonderful to hear! I'm so glad that perspective resonated with you and provided some comfort. It's powerful to remember that your inner strength and perseverance can lead to positive outcomes, even when things feel incredibly tough.




"That's wonderful to hear! I'm so glad that perspective resonated with you and provided some comfort. It's powerful to remember that your inner strength and perseverance can lead to positive outcomes, even when things feel incredibly tough."

In [190]:
# Step 4: Test that the pattern is used next time (same trigger)
await run_query(
    "I'm having one of those days where everything feels overwhelming and hard again.",
    user_id="alice123",
    session_id="session_alice_therapy"
)


USER: I'm having one of those days where everything feels overwhelming and hard again.

ASSISTANT: It takes strength to acknowledge these feelings, and I'm here to support you. Would you like to talk more about what's making you feel overwhelmed, or would you prefer a different perspective right now?




"It takes strength to acknowledge these feelings, and I'm here to support you. Would you like to talk more about what's making you feel overwhelmed, or would you prefer a different perspective right now?"

In [192]:
# Step 5: Test disliking a response (to show both patterns being tracked)
await run_query(
    "I'm feeling so stressed about work. Can you help?",
    user_id="alice123",
    session_id="session_alice_therapy2"
)


USER: I'm feeling so stressed about work. Can you help?

ASSISTANT: I understand that work-related stress can be really overwhelming. It's tough when you feel that pressure mounting. Remember to take a deep breath. Is there anything specific about work that's causing this stress? Perhaps we can break it down into smaller pieces.




"I understand that work-related stress can be really overwhelming. It's tough when you feel that pressure mounting. Remember to take a deep breath. Is there anything specific about work that's causing this stress? Perhaps we can break it down into smaller pieces."

In [193]:
# Step 6: User dislikes the response
await run_query(
    "That doesn't really help me. I need something different.",
    user_id="alice123",
    session_id="session_alice_therapy2"
)


USER: That doesn't really help me. I need something different.

ASSISTANT: I understand that work-related stress can be really overwhelming. It's tough when you feel that pressure mounting. Remember to take a deep breath. Is there anything specific about work that's causing this stress? Perhaps we can break it down into smaller pieces.




"I understand that work-related stress can be really overwhelming. It's tough when you feel that pressure mounting. Remember to take a deep breath. Is there anything specific about work that's causing this stress? Perhaps we can break it down into smaller pieces."

## 10. View Stored Data

Let's check what data has been stored:

In [197]:
print("=" * 60)
print("STORED USER MEMORY:")
print("=" * 60)
print(json.dumps(user_memory_store, indent=2))

print("\n" + "=" * 60)
print("SCHEDULED CALENDAR EVENTS:")
print("=" * 60)
print(json.dumps(calendar_events, indent=2))

print("\n" + "=" * 60)
print("THERAPEUTIC RESPONSE PATTERNS (What Works/Doesn't Work):")
print("=" * 60)
print(json.dumps(therapeutic_patterns, indent=2))

STORED USER MEMORY:
{
  "alice123": {
    "preferences": {},
    "history": [],
    "personal_details": {
      "name": "Alice"
    },
    "interests": [
      "mindfulness and meditation"
    ]
  }
}

SCHEDULED CALENDAR EVENTS:
[
  {
    "id": 1,
    "user_id": "alice123",
    "title": "Meditation session",
    "date": "2024-04-01",
    "time": "07:00",
    "description": "",
    "created_at": "2025-12-01T03:06:54.431811"
  }
]

THERAPEUTIC RESPONSE PATTERNS (What Works/Doesn't Work):
{
  "alice123": {
    "triggers": {
      "life is hard": {
        "liked_responses": [
          {
            "response": "It sounds like you're going through a really tough time, and it's completely understandable to feel that way when things seem to be piling up. I'm here to listen and support you. Can you tell me a bit more about what's been going wrong? Sometimes just talking about it can help ease the burden.",
            "notes": "",
            "timestamp": "2025-12-01T03:05:14.895726"
       

## 11. Advanced: Agent State Tracking

In [ ]:
# Create a custom query with state tracking
async def run_query_with_state(query: str, user_id: str = "user123", session_id: str = "session456"):
    """Run a query and track tool context state."""
    print(f"\nUSER: {query}\n")
    
    # Create session if it doesn't exist
    try:
        await session_service.get_session(
            app_name="personal_psychologist_app",
            user_id=user_id,
            session_id=session_id
        )
    except:
        # Session doesn't exist, create it
        await session_service.create_session(
            app_name="personal_psychologist_app",
            user_id=user_id,
            session_id=session_id,
            state={}
        )
    
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    tool_calls = []
    responses = []
    
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=content
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    responses.append(part.text)
                    print(f"[{event.author}]: {part.text}")
                elif hasattr(part, 'function_call') and part.function_call:
                    tool_calls.append({
                        'name': part.function_call.name,
                        'args': dict(part.function_call.args)
                    })
    
    return {
        'tool_calls': tool_calls,
        'responses': responses
    }

# Test with state tracking
result = await run_query_with_state(
    "Schedule a therapy session for next Monday at 3 PM and remember that I prefer afternoon appointments.",
    user_id="alice123",
    session_id="session_alice_1"
)

print("\n=" * 30)
print("SUMMARY:")
print(f"Tool calls made: {len(result['tool_calls'])}")
print(f"Tools used: {[tc['name'] for tc in result['tool_calls']]}")

## 12. Export Agent Configuration

Export the agent structure for documentation:

In [ ]:
def export_agent_structure(agent, level=0):
    """Recursively export agent structure."""
    indent = "  " * level
    print(f"{indent}📋 Agent: {agent.name}")
    print(f"{indent}   Model: {agent.model}")
    print(f"{indent}   Description: {agent.description}")
    
    if hasattr(agent, 'tools') and agent.tools:
        print(f"{indent}   Tools: {[tool.__name__ if callable(tool) else str(tool) for tool in agent.tools]}")
    
    if hasattr(agent, 'sub_agents') and agent.sub_agents:
        print(f"{indent}   Sub-agents:")
        for sub_agent in agent.sub_agents:
            export_agent_structure(sub_agent, level + 1)
    print()

print("\n" + "=" * 60)
print("AGENT SYSTEM STRUCTURE:")
print("=" * 60 + "\n")
export_agent_structure(root_agent)

## 13. Cleanup and Reset

In [ ]:
# Clear all stored data
def reset_system():
    """Reset all stored data."""
    global user_memory_store, calendar_events
    user_memory_store = {}
    calendar_events = []
    print("✓ System reset complete. All data cleared.")

# Uncomment to reset:
# reset_system()

## Summary

This notebook demonstrates a complete hierarchical multi-agent system built with Google ADK:

### Agents

- **Memory Agent**: Manages user context, preferences, and personal details
- **Calendar Agent**: Handles scheduling and reminders (with MCP Google Calendar integration option)
- **Feedback Agent**: Processes user feedback for continuous improvement
- **Therapeutic Agent**: 🆕 Provides supportive responses and **learns what works** for each user
- **Root Agent**: Coordinates all agents and provides empathetic support

### Key Features

The system uses:
- ✅ Custom tools with state management
- ✅ Hierarchical agent delegation
- ✅ Session-based conversation continuity
- ✅ In-memory data persistence
- ✅ **MCP Integration** (Model Context Protocol) for Google Calendar
- ✅ **Therapeutic Pattern Learning** - tracks trigger/response patterns to learn what helps each user

### Therapeutic Pattern Tracking

The therapeutic agent implements a unique learning system:
1. When user shares an emotional struggle → provides supportive response
2. When user asks for another option → provides alternative perspective
3. When user says they like a response → saves as "preferred pattern"
4. When user dislikes a response → saves to avoid similar responses
5. Future interactions → uses learned patterns for personalized support

Example patterns tracked:
- **Trigger**: "life is hard"
- **Liked Response**: "miracles happen for those who never quit"
- **Disliked Response**: (anything user rejected)

## Next Steps

1. ✅ Add persistent storage (database integration) - use MCP for database access
2. ✅ Integrate real Google Calendar API - MCP integration ready
3. Add more sophisticated NLP for intent detection
4. Implement evaluation metrics for agent performance
5. Deploy as a web service using FastAPI
6. Expand therapeutic pattern matching with embeddings for better trigger similarity detection